In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import pandas as pd
from pathlib import Path
import src.config
from deltalake import DeltaTable

In [ ]:
wh_allroles = DeltaTable(src.config.BRONZE / "wh_allroles").to_pandas()


In [ ]:
ALERTS_CURRENT_FILE = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Credit Loan Review\Alerts\Production\Output\alerts.xlsx")
alerts = pd.read_excel(ALERTS_CURRENT_FILE)

In [ ]:
alerts

In [ ]:
guar = wh_allroles[wh_allroles['acctrolecd'] == 'GUAR'].copy()

In [ ]:
guar = guar[~(guar['persnbr'].isnull())].copy()

In [ ]:
guar

In [ ]:
guar['persnbr'] = guar['persnbr'].astype('Int64').astype(str)


In [ ]:
merged_eligible = alerts.merge(guar, on='acctnbr', how='left')

In [ ]:
merged_eligible

In [ ]:
PERMISSION_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Credit Loan Review\Alerts\Production\assets\xactus\pfs_permission.csv")
permission = pd.read_csv(PERMISSION_PATH)

In [ ]:
permission

In [ ]:
permission['persnbr'] = permission['persnbr'].astype(str)

In [ ]:
check = merged_eligible.merge(permission, how='outer', on='persnbr', indicator=True)

In [ ]:
check

In [ ]:
no_permission = check[check['_merge'] == 'left_only'].copy()
no_permission = no_permission[[
    'acctnbr',
    'ownername',
    'product',
    'loanofficer',
    'inactivedate',
    'persnbr'
]].copy()

In [ ]:
# Get bronze permission tables to get names
wh_pers = DeltaTable(src.config.BRONZE / "wh_pers").to_pandas()
assert wh_pers['persnbr'].is_unique, "Persnbr not unique in wh_pers"

In [ ]:
wh_pers

In [ ]:
wh_pers = wh_pers[['persnbr','perssortname','age']].copy()

In [ ]:
wh_pers['persnbr'] = wh_pers['persnbr'].astype(str)
no_permission_detail = no_permission.merge(wh_pers, on='persnbr', how='left')

In [ ]:
no_permission_detail = no_permission_detail[~(no_permission_detail['persnbr'].isnull())].copy()

In [ ]:
no_permission_detail

In [ ]:
no_permission_detail['acctnbr'] = no_permission_detail['acctnbr'].astype('Int64').astype(str)

In [ ]:
no_permission_detail

In [ ]:
no_permission_detail

In [ ]:
guar